# 1) Importing the datasets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [5]:
data_yelp = pd.read_csv('/yelp_labelled.txt', sep='\t',header=None)

In [6]:
data_yelp.head()
# review and sentiment
# 0-Negative, 1-Positive for positive review

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
# Assign column names
columan_name = ['Review', 'Sentiment']
data_yelp.columns = columan_name

In [8]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [9]:
data_yelp.shape
# 1000 rows (reviews), 2 columns (Sentiments)

(1000, 2)

In [10]:
data_amazon = pd.read_csv('/amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()
# review and sentiment
# 0-Negative, 1-Positive for positive review

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [11]:
data_amazon.columns = columan_name

In [12]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [13]:
data_amazon.shape

(1000, 2)

In [15]:
data_imdb = pd.read_csv('/imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [16]:
data_imdb.columns = columan_name

In [17]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [18]:
data_imdb.shape

(748, 2)

In [19]:
# Append all the data in a single dataframe

In [20]:
data = data_yelp.append([data_amazon, data_imdb],ignore_index=True)

In [21]:
data.shape

(2748, 2)

In [22]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [23]:
# check distribution of sentiments

In [24]:
data['Sentiment'].value_counts()

# 1346 positive reviews
# 1362 Negative reviews

1    1386
0    1362
Name: Sentiment, dtype: int64

In [25]:
# check for null values
data.isnull().sum()

# no null values in the data

Review       0
Sentiment    0
dtype: int64

In [26]:
x = data['Review']
y = data['Sentiment']

# 2) Data Cleaning

In [27]:
# here we will remove stopwords, punctuations
# as well as we will apply lemmatization

## Create a function to clean the data

In [28]:
import string

In [29]:
punct = string.punctuation

In [30]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [31]:
from spacy.lang.en.stop_words import STOP_WORDS

In [32]:
stopwords = list(STOP_WORDS) # list of stopwords

In [33]:
# creating a function for data cleaning

In [34]:
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
 
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [35]:
# if root form of that word is not pronoun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, because there is no lemma for proper noun

In [36]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")
# stopwords and punctuations removed

['hello', 'beautiful', 'day', 'outside']

## Vectorization Feature Engineering (TF-IDF)

In [37]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [38]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
# tokenizer=text_data_cleaning, tokenization will be done according to this function

In [39]:
classifier = LinearSVC()

# 3) Train the model

## Splitting the dataset into the Train and Test set

In [40]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [41]:
x_train.shape, x_test.shape
# 2198 samples in training dataset and 550 in test dataset

((2198,), (550,))

In [42]:
x_train.head()

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
Name: Review, dtype: object

## Fit the x_train and y_train

In [43]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])
# it will first do vectorization and then it will do classification

In [44]:
clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7fbffdb6fe60>)),
                ('clf', LinearSVC())])

In [45]:
# in this we don't need to prepare the dataset for testing(x_test)

# 4) Predict the Test set results

In [46]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [47]:
y_pred = clf.predict(x_test)

In [48]:
# confusion_matrix
confusion_matrix(y_test, y_pred)

array([[201,  78],
       [ 50, 221]])

In [49]:
# classification_report
print(classification_report(y_test, y_pred))
# we are getting almost 77% accuracy

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [50]:
accuracy_score(y_test, y_pred)
# 76% accuracy

0.7672727272727272

In [51]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])
# output is 1, that means review is positive

array([1])

In [52]:
clf.predict(["It's hard to learn new things!"])
# output is 0, that means review is Negative

array([0])